In [1]:
pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 KB 6.1 MB/s eta 0:00:00


In [21]:

import networkx as nx
import pandas as pd
import os
import sys
import stellargraph as sg
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE
from collections import Counter
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np

Loading the cora dataset

In [3]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, node_subjects = dataset.load()
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 5429

 Node types:
  paper: [2708]
    Features: float32 vector, length 1433
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5429]
        Weights: all 1 (default)
        Features: none


our goal is to predict the categories/node subjects

In [4]:
set(node_subjects)

{'Case_Based',
 'Genetic_Algorithms',
 'Neural_Networks',
 'Probabilistic_Methods',
 'Reinforcement_Learning',
 'Rule_Learning',
 'Theory'}

splitting the data
150 nodes for training
500 nodes for validation
2058 nides for validation

In [5]:
train_subjects, test_subjects = model_selection.train_test_split(node_subjects, train_size=140, test_size=None, stratify=node_subjects)

val_subjects, test_subjects = model_selection.train_test_split(test_subjects, train_size=500, test_size=None, stratify=test_subjects)

cheking sample sizes for weighted cross antropy loss function in training

In [6]:
from collections import Counter

Counter(train_subjects)

Counter({'Case_Based': 16,
         'Genetic_Algorithms': 22,
         'Theory': 18,
         'Neural_Networks': 42,
         'Reinforcement_Learning': 11,
         'Probabilistic_Methods': 22,
         'Rule_Learning': 9})

categorigal featuree vectores to numrical feature vectors with one-hot embedding

In [7]:
target_encoding = preprocessing.LabelBinarizer()
train_targets = target_encoding.fit_transform(train_subjects)
test_targets = target_encoding.transform(test_subjects)
val_targets = target_encoding.transform(val_subjects)

# Creating the GraphSAGE model 

generator feeds data from graph into model, A GraphSAGENodeGenerator object sends the node features in sampled subgraphs to Keras

In [8]:
#batch size = 50 (num of nodes to sample at each level)
#two-level model with 10 nodes in first layer and 5 nodes in second layer
generator = GraphSAGENodeGenerator(G, 50, [10,5])

In [9]:
#iterator over training,validation and test nodes with shuffle to improve training
train_gen = generator.flow(train_subjects.index, train_targets, shuffle=True)
val_gen = generator.flow(val_subjects.index, val_targets)
test_gen = generator.flow(test_subjects.index, test_targets)

In [10]:
#specifying the model
#layer_sizes:hidden feature sizes of each layer in the model(32-dimensional hidden node features at each layer)
graphsage_model = GraphSAGE(layer_sizes=[32, 32], generator=generator, bias=True, dropout=0.5,)

specifying input and output

In [11]:
x_inp, x_out = graphsage_model.in_out_tensors()

prediction = layers.Dense(units=train_targets.shape[1], activation="softmax")(x_out)

# Training the model

In [12]:
#crearing Keras model with inputs from the graph_model and outputs are the predictions from the softmax layer
model = Model(inputs=x_inp, outputs=prediction)
model.compile(
    optimizer=optimizers.Adam(lr=0.005),
    loss=losses.categorical_crossentropy,
    metrics=["acc"],
)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
#training
history = model.fit(
    train_gen, epochs=20, validation_data=test_gen, verbose=2, shuffle=False
)

Epoch 1/20
3/3 - 5s - loss: 1.9459 - acc: 0.1643 - val_loss: 1.7850 - val_acc: 0.3235 - 5s/epoch - 2s/step
Epoch 2/20
3/3 - 2s - loss: 1.7171 - acc: 0.3643 - val_loss: 1.7193 - val_acc: 0.3100 - 2s/epoch - 652ms/step
Epoch 3/20
3/3 - 3s - loss: 1.6183 - acc: 0.3714 - val_loss: 1.6474 - val_acc: 0.3341 - 3s/epoch - 940ms/step
Epoch 4/20
3/3 - 2s - loss: 1.5095 - acc: 0.4857 - val_loss: 1.5475 - val_acc: 0.4550 - 2s/epoch - 662ms/step
Epoch 5/20
3/3 - 3s - loss: 1.4013 - acc: 0.7071 - val_loss: 1.4557 - val_acc: 0.6257 - 3s/epoch - 943ms/step
Epoch 6/20
3/3 - 2s - loss: 1.3143 - acc: 0.8929 - val_loss: 1.3863 - val_acc: 0.7012 - 2s/epoch - 643ms/step
Epoch 7/20
3/3 - 3s - loss: 1.2353 - acc: 0.9000 - val_loss: 1.3176 - val_acc: 0.7369 - 3s/epoch - 947ms/step
Epoch 8/20
3/3 - 2s - loss: 1.1311 - acc: 0.9643 - val_loss: 1.2641 - val_acc: 0.7577 - 2s/epoch - 661ms/step
Epoch 9/20
3/3 - 3s - loss: 1.0722 - acc: 0.9500 - val_loss: 1.2154 - val_acc: 0.7592 - 3s/epoch - 942ms/step
Epoch 10/20
3

In [14]:
sg.utils.plot_history(history)

evaluate the test set

In [16]:
test_metrics = model.evaluate(test_gen)

  

42/42 [==============================] - 2s 47ms/step - loss: 0.8812 - acc: 0.7921


# Predictions with model

prediction for all nodes

In [17]:
all_nodes = node_subjects.index
all_mapper = generator.flow(all_nodes)
all_predictions = model.predict(all_mapper)

55/55 [==============================] - 3s 41ms/step


all_predictions is the result of softmax layer so we inverse_transform to get categorical data

In [18]:
node_predictions = target_encoding.inverse_transform(all_predictions)

examples

In [20]:
df = pd.DataFrame({"Predicted": node_predictions, "True": node_subjects})
df.head(20)

Predicted                    True
31336                    Theory         Neural_Networks
1061127           Rule_Learning           Rule_Learning
1106406  Reinforcement_Learning  Reinforcement_Learning
13195    Reinforcement_Learning  Reinforcement_Learning
37879     Probabilistic_Methods   Probabilistic_Methods
1126012  Reinforcement_Learning   Probabilistic_Methods
1107140  Reinforcement_Learning                  Theory
1102850   Probabilistic_Methods         Neural_Networks
31349     Probabilistic_Methods         Neural_Networks
1106418                  Theory                  Theory
1123188         Neural_Networks         Neural_Networks
1128990         Neural_Networks      Genetic_Algorithms
109323    Probabilistic_Methods   Probabilistic_Methods
217139               Case_Based              Case_Based
31353                    Theory         Neural_Networks
32083           Neural_Networks         Neural_Networks
1126029  Reinforcement_Learning  Reinforcement_Learning
1118017         Neural_Networks         Neural_Networks
49482           Neural_Networks         Neural_Networks
753265                   Theory         Neural_Networks

# Node embeddings


evaluating node embeddings (dim reduction with pca) by visualizing them with their true labels as colors. since we want to evaluate the embeddings we need the output of the first layer (output is embedding instead of predicted class in new model)
+previously trained weights are kept in the new model

In [22]:
embedding_model = Model(inputs=x_inp, outputs=x_out)
emb = embedding_model.predict(all_mapper)
X = emb
y = np.argmax(target_encoding.transform(node_subjects), axis=1)
if X.shape[1] > 2:
    transform = TSNE  # PCA

    trans = transform(n_components=2)
    emb_transformed = pd.DataFrame(trans.fit_transform(X), index=node_subjects.index)
    emb_transformed["label"] = y
else:
    emb_transformed = pd.DataFrame(X, index=node_subjects.index)
    emb_transformed = emb_transformed.rename(columns={"0": 0, "1": 1})
    emb_transformed["label"] = y
alpha = 0.7

fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(
    emb_transformed[0],
    emb_transformed[1],
    c=emb_transformed["label"].astype("category"),
    cmap="jet",
    alpha=alpha,
)
ax.set(aspect="equal", xlabel="$X_1$", ylabel="$X_2$")
plt.title(
    "{} visualization of GraphSAGE embeddings for cora dataset".format(transform.__name__)
)
plt.show()

55/55 [==============================] - 5s 78ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
